## TensorFlow in SageMaker
Based on https://docs.aws.amazon.com/sagemaker/latest/dg/tf-examples.html and https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_abalone_age_predictor_using_keras/abalone.py

For more information on using TensorFlow estimators with SageMaker see https://github.com/aws/sagemaker-python-sdk#tensorflow-sagemaker-estimators

SageMaker allows for training of models across multiple nodes and can host the resulting models as a RESTful endpoint for consumption.  SageMaker can also take a custom TensorFlow body of code and execute it using a SageMaker container (no need to bring-your-own-algo in your own container).  

This notebook uses the example cited above to adapt the Tensorflow code from https://github.com/jamesandersen/aws-machine-learning-demo and train / host the code in SageMaker.

In [ ]:
! cat sage_model.py

In [ ]:
# Initialize variables
from sagemaker import get_execution_role

# Bucket location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://jasbarto-sagemaker-eval/loan_grade/tensorflow/customcode'

# Bucket location where results of model training are saved.
model_artifacts_location = 's3://jasbarto-sagemaker-eval/loan_grade/tensorflow/artifacts'

# IAM execution role that gives SageMaker access to resources in your AWS account.
role = get_execution_role()

In [ ]:
# Train the model, providing the custom Tensorflow code
from sagemaker.tensorflow import TensorFlow

loan_grade_estimator = TensorFlow(
                            base_job_name = 'loan-grade-tensorflow', # provide a base name for the training job
                            entry_point='sage_model.py', # pass a reference to the Python source file for the model
                            role = role, # use the same role defined for this notebook
                            output_path = model_artifacts_location, # where to deposit the resulting trained model
                            code_location = custom_code_upload_location, # where to upload the Python script in S3
                            train_instance_count = 1, # how many instances to use to train the model
                            train_instance_type = 'ml.c4.xlarge',
                            training_steps = 250000, # how many steps of training to perform
                            evaluation_steps = None, # evaluate the model until evaluation data is consumed
                            hyperparameters = {
                                'learning_rate': 0.001, # parameter passed to Tensorflow optimization
                                'min_eval_frequency': 1000, # 
                                'save_checkpoints_secs': 3
                                'save_summary_steps': 1000
                            }
                        )

## Train the model

SageMaker enables familiar tools like TensorBoard to be used with the model training process.  The ``fit`` function below runs an instance of TensorBoard which can be accessed at [/proxy/6006/](/proxy/6006/).

In [ ]:
# start the training
%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://jasbarto-sagemaker-eval/loan_grade_data'

loan_grade_estimator.fit(train_data_location, run_tensorboard_locally = True)

In [ ]:
%%time
# deploy the trained model

training_job_name = loan_grade_estimator.latest_training_job.name
print ("Deploying training job: {}".format (training_job_name))

loan_grade_predictor = loan_grade_estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

## Evaluate the effectiveness of the deployed, trained model

In [ ]:
import sage_model
import tensorflow as tf
import numpy as np

# read the sample data set and select a record for submission to the endpoint
sage_model.read_csv_data ('.')

predict_features = sage_model.eval_data['features'][1337].tolist ()
predict_label = sage_model.eval_data['labels'][1337].tolist ()

print ("Features length {}, type {}, data {}".format (len(predict_features), type(predict_features), predict_features))
print ("True label: {}".format (predict_label))

In [ ]:
import pprint as pp

# format the features record into a tensor proto suitable for TensorFlow submission and send it to the endpoint
tensor_proto = tf.make_tensor_proto(values=np.asarray (predict_features), shape=[1, len(predict_features)], dtype=tf.float32)
loan_grade = loan_grade_predictor.predict (tensor_proto)
pp.pprint (loan_grade)

In [ ]:
# do this times a lot, generate a confusion matrix to show the results
predict_limit = 1000
features = sage_model.eval_data['features'][:predict_limit].tolist ()
labels = np.argmax (sage_model.eval_data['labels'][:predict_limit].tolist (), axis = 1)
predictions = []

for f in features:
    tensor_proto = tf.make_tensor_proto (values = np.asarray (f), shape = [1, len(f)], dtype = tf.float32)
    pred_grade = loan_grade_predictor.predict (tensor_proto)
    predictions.append (int (pred_grade['outputs']['classes']['int64Val'][0]))

In [ ]:
import matplotlib.pyplot as plt
import itertools 
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score (labels, predictions)
cf_matrix = confusion_matrix (labels, predictions)

plt.imshow (cf_matrix, interpolation = 'nearest', cmap = plt.cm.Oranges)
plt.title ('Endpoint Confusion Matrix')
plt.colorbar ()
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
tick_marks = np.arange (len (classes))
plt.yticks (tick_marks, classes)
plt.xticks (tick_marks, classes)
plt.ylabel ('True label')
plt.xlabel ('Predicted label')

thresh = cf_matrix.max() / 2.
for i, j in itertools.product(range(cf_matrix.shape[0]), range(cf_matrix.shape[1])):
        plt.text(j, i, format(cf_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cf_matrix[i, j] > thresh else "black")

plt.show ()
print ("Overall accuracy over {} samples: {}".format (predict_limit, accuracy))

In [ ]:
# finally delete the endpoint
from sagemaker import Session
Session().delete_endpoint (loan_grade_predictor.endpoint)